# Import

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import seaborn as sns
import matplotlib.pyplot as plt



Bad key "text.kerning_factor" on line 4 in
/Users/aaron/anaconda3/envs/anaconda3_base/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [56]:
%autosave 60
%matplotlib inline


Autosaving every 60 seconds


# Make example data csv for this notebook

In [52]:
import random

def make_example_data(n, filename='test_data_from_notebook.csv'):
    IDs = np.arange(n)
    top_speed = np.abs(np.random.randn(n)) * 200
    
    
    df = pd.DataFrame({
        'ID' : IDs.astype(int), 
        'top speed': top_speed,
        'destination': np.random.choice(['Mars', 'Moon', 'Jupiter', ''], size=n),
        'Max Q': np.sqrt(top_speed)
        })
    
    df.ID = df.astype({'ID': 'int32'})
    
    # randomly set 10% of cells to null.
    df = df.mask(np.random.random(df.shape) < .05)
    
    # save to file.
    df.to_csv(filename, index=False)
    
    print("Just created '{}' of length {}".format(
        filename, n))
    
    display(df.head())
    
    
make_example_data(n=50)



Just created 'test_data_from_notebook.csv' of length 50


,ID,top speed,destination,Max Q
0,0,73.845304,Mars,8.593329
1,NaN,65.887660,Mars,8.117121
2,2,256.321479,NaN,16.010043
3,3,110.864038,Moon,10.529199
4,4,151.866393,Jupiter,12.323408


# Import CSV

In [53]:
my_df = pd.read_csv('test_data_from_notebook.csv')

# Useful:
#    parse_dates=['fbdate_oconf', 'farrive', 'fstart', 'ffinish'],
#    infer_datetime_format=True)
#    missing_values = []

my_df.head()

,ID,top speed,destination,Max Q
0,0.0,73.845304,Mars,8.593329
1,NaN,65.887660,Mars,8.117121
2,2.0,256.321479,NaN,16.010043
3,3.0,110.864038,Moon,10.529199
4,4.0,151.866393,Jupiter,12.323408


# EDA: nulls, boxplots ... 

In [54]:
def describe_nulls(df, show_nulls_table=False):    
    print("Total rows: \n\t{:,}".format(len(df)))
    
    rows_with_any_NA = len(df[df.isnull().any(axis=1)])
    print("Rows; percent of rows that have *at least 1* missing value: \n\t{:,}; {}%".format(
        rows_with_any_NA, 100 * round(rows_with_any_NA / len(df), 2)))
    
    
    if show_nulls_table:
        nunique = df.nunique() # just compute this once.
    
        display(pd.DataFrame({
            'dtype': df.dtypes,
            'Number of nulls': df.isnull().sum(),
            '% null': 100 * round(df.isnull().sum() / len(df), 4),
            'Distinct Values': nunique,
            '% Distinct Values': 100 * round(nunique / len(df), 4),
        }))

    return

describe_nulls(my_df, show_nulls_table=True)

Total rows: 
	50
Rows; percent of rows that have *at least 1* missing value: 
	23; 46.0%


,dtype,Number of nulls,% null,Distinct Values,% Distinct Values
ID,float64,3,6.0,47,94.0
top speed,float64,2,4.0,48,96.0
destination,object,16,32.0,3,6.0
Max Q,float64,4,8.0,46,92.0


## Pandas: SQL-like and other common functions

* groupby()
* nunique()
* df.cat.sum()

# Misc: Mapping, ... 

In [ ]:
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs

def pt_plot_USA(df, latname, lonname, hue):
    contiguous_usa = gpd.read_file(gplt.datasets.get_path('contiguous_usa'))

    ax = gplt.polyplot(
        contiguous_usa,
        zorder=-1,
        linewidth=1,
        projection=gcrs.AlbersEqualArea(),
        edgecolor='white',
        facecolor='lightgray',
        figsize=(17, 8))

    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(
            df[lonname], df[latname]))

    gplt.pointplot(
        gdf,
        hue=hue,
        legend=True,
        cmap='viridis',
        alpha=0.8,
        ax = ax)
    
    plt.title('Truckloads, colored by total weight', fontsize=25)


#pt_plot_USA(df=lower_48, latname='lat', lonname='lon', hue='total truck weight')